# Process BenchExec Experiments

This notebook post-processes the results obtained from [Benchexec](https://github.com/sosy-lab/benchexec) tables.

The Benchexec tables contain sets of (repeated) stat columns, one per run set. Each run set can represent a specific solver. Each row is a problem instance, called a _task_ in Benchexec.

This set-up is not very useful to do plotting or stat tables. Instead, we would like to re-shape and have a special column that records the solver, and  so one set of stat columns (with no repetition).

This network will then produce two tables:

1. A flat table of stats, that can be used for further analysis and plotting, with the solver being recorded in a new column.
2. A coverage table per domain and solver, typically reported in papers.

Sebastian Sardina 2024 - ssardina@gmail.com

In [15]:
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
import glob
import re

# sys.dont_write_bytecode = True  # prevent creation of .pyc files

CSV_FOLDER = "stats/ecai23-redo-benchexec-jul24/"
NAME_EXPERIMENT = "cfond"

## 1. Flatten Benchexec CSV tables

Collect all CSV benchexec table result files under `CSV_FOLDER` folder.

In [16]:
# files produced when tables come from single benchexec run (one .xml.bz2 file)
benchexec_csv_files = glob.glob(os.path.join(CSV_FOLDER, "**", "benchmark-*.csv"), recursive=True)

# files produced when tables come from multiple benchexec runs (many .xml.bz2 files)
benchexec_csv_files += glob.glob(
    os.path.join(CSV_FOLDER, "**", "results.*.table.csv"), recursive=True
)

print(f"CSV benchexec table files found to extract and combine stats in folder {CSV_FOLDER}:")
benchexec_csv_files

CSV benchexec table files found to extract and combine stats in folder stats/ecai23-redo-benchexec-jul24/:


['stats/ecai23-redo-benchexec-jul24/paladinus-19-07-24/benchmark-paladinus.2024-07-19_07-50-23.results.paladinus.FOND.csv',
 'stats/ecai23-redo-benchexec-jul24/prp-19-07-24/benchmark-prp.2024-07-19_17-59-23.results.prp.FOND.csv',
 'stats/ecai23-redo-benchexec-jul24/cfondasp-23-08-24/results.2024-09-02_07-34-19.table.csv',
 'stats/ecai23-redo-benchexec-jul24/fondsat-21-07-24/results.2024-07-29_15-40-20.table.csv',
 'stats/ecai23-redo-benchexec-jul24/cfondasp-02-09-24/results.2024-09-10_13-39-17.table.csv']

Next, **load all CSV files into a single dataframe**. As explained above, each CSV file may include results over many _run sets_, with each having its own (set of) columns.

Each row is the result of a _task_ in the experiment, and every run set has its columns stats for such task. Presumably sets of columns share the same schema.

We then need reshape this structure to have just one set of columns and a new column identifying the run set. So each original row will be expanded into many rows, one per run set.

The first three lines contain header:

1. First line contains the **_tool_** used. It starts with `tool` followed by the name of the tool repeated multiple times (to match no. of columns).
2. Second line contains the **_runs_** of the experiment. It starts with `run set` and then sets of columns with the name of the runs.
3. Third line contains the **_stats_** column names repeated per run set. First column is for name of the task.

In [18]:
RENAME_COLS = {"benchmarks/benchexe/tasks/": "id", "cputime (s)": "cputime", "walltime (s)": "walltime", "memory (MB)": "memory_mb"}

def get_meta_csv(file):
    """Given a benchexec CSV file, extract the runs (e.g., prp, prp_inv) and how many columns per run"""
    with open(file, "r") as f:
        # first line contains the tool used (repeated one per column needed); e.g.,  PP-FOND
        tools_header = f.readline().split()[1:]
        # second line contains the run/solvers used in the experiment (e.g., prp, prp_inv) and starts with "run set" to be ignored - will have duplicates, one per stat col in run
        runs_header = f.readline().split()[2:]

    # get the run names (e.g., prp, prp_inv), drop duplicates but preserve order (order of columns in CSV)
    # OBS: cannot just use set() as that will change the order of columns in CSV
    runs = list(dict.fromkeys(runs_header))

    # no of stat columns per run
    no_cols = int(len(runs_header) / len(runs))

    return runs, no_cols


# iterate through each CSV file
dfs = []
for f in benchexec_csv_files:
    runs, no_cols = get_meta_csv(f)
    print(f"Runs in file {f}: {runs} with {no_cols} stat columns")

    # go over each set of run columns (a CSV table file may contain several runs, each with the same columns)
    for k, r in enumerate(list(runs)):
        col_idx = [0] + list(range(k*no_cols + 1, k*no_cols + no_cols + 1))
        print(f"\t Extracting run '{r}' in columns: {col_idx}")

        # read the CSV file from line 3+ (line 3 is header)
        df = pd.read_csv(f, delimiter="\t", skiprows=2, usecols=col_idx)
        df.rename(columns=lambda x: x.split('.')[0], inplace=True)

        df.columns.values[0] = "task"
        # df.rename(columns={df.columns[1]: "task"})

        # populate column run with name of run-solver r
        df.insert(1, 'run', r)
        dfs.append(df)

df_csv = pd.concat(dfs).reset_index(drop=True)

df_csv.rename(columns=RENAME_COLS, inplace=True)

print("Runs found:", df_csv["run"].unique())
for s in df_csv["run"].unique():
    print(s)
# df.set_index("task", inplace=True)


# df_csv.query("task == 'acrobatics_01.yml' and run == 'cfondasp1-reg.FOND'")
df_csv.query("run == 'cfondasp2-reg.FOND'")
# df_csv

Runs in file stats/ecai23-redo-benchexec-jul24/paladinus-19-07-24/benchmark-paladinus.2024-07-19_07-50-23.results.paladinus.FOND.csv: ['paladinus.FOND'] with 6 stat columns
	 Extracting run 'paladinus.FOND' in columns: [0, 1, 2, 3, 4, 5, 6]
Runs in file stats/ecai23-redo-benchexec-jul24/prp-19-07-24/benchmark-prp.2024-07-19_17-59-23.results.prp.FOND.csv: ['prp.FOND'] with 6 stat columns
	 Extracting run 'prp.FOND' in columns: [0, 1, 2, 3, 4, 5, 6]
Runs in file stats/ecai23-redo-benchexec-jul24/cfondasp-23-08-24/results.2024-09-02_07-34-19.table.csv: ['cfondasp1-fsat.FOND', 'cfondasp1-reg.FOND'] with 6 stat columns
	 Extracting run 'cfondasp1-fsat.FOND' in columns: [0, 1, 2, 3, 4, 5, 6]
	 Extracting run 'cfondasp1-reg.FOND' in columns: [0, 7, 8, 9, 10, 11, 12]
Runs in file stats/ecai23-redo-benchexec-jul24/fondsat-21-07-24/results.2024-07-29_15-40-20.table.csv: ['fondsat-minisat.FOND', 'fondsat-glucose.FOND'] with 6 stat columns
	 Extracting run 'fondsat-minisat.FOND' in columns: [0, 1,

,task,run,status,cputime,walltime,memory_mb,planner_time,policy_size
4130,acrobatics_01.yml,cfondasp2-reg.FOND,true,0.824157,0.590535,47.054848,0.093831,5
4131,acrobatics_02.yml,cfondasp2-reg.FOND,true,0.906004,0.671717,47.108096,0.164438,8
4132,acrobatics_03.yml,cfondasp2-reg.FOND,true,7.466023,4.461721,66.293760,3.936452,16
4133,acrobatics_04.yml,cfondasp2-reg.FOND,true,6374.627300,3196.744161,220.270592,3196.186724,32
4134,acrobatics_05.yml,cfondasp2-reg.FOND,TIMEOUT (false),14402.044700,7219.618570,366.690304,-1.000000,-1
...,...,...,...,...,...,...,...,...
4715,zenotravel_11.yml,cfondasp2-reg.FOND,TIMEOUT (false),14400.294209,14404.139162,863.870976,-1.000000,-1
4716,zenotravel_12.yml,cfondasp2-reg.FOND,TIMEOUT (false),14400.300153,14400.606881,846.143488,-1.000000,-1
4717,zenotravel_13.yml,cfondasp2-reg.FOND,TIMEOUT (false),14400.297785,14401.525199,762.023936,-1.000000,-1
4718,zenotravel_14.yml,cfondasp2-reg.FOND,TIMEOUT (false),14400.613112,14402.463129,924.254208,-1.000000,-1


We next **enrich** the dataframe with the following derived columns:

* domain
* instance
* solver
* solved (boolean)

In [32]:
def get_benchmark_labels(task_name):
    """From the task description name (e.g., acrobatics_01.yml), extract the benchmark labels, like domain, instance"""
    regex = r"(.+)_([0-9]+)\.yml"

    match = re.match(regex, task_name)
    if match:
        # print(match.groups())
        domain = match.group(1)
        instance = match.group(2)
    else:
        print("Problem extracting labels from task name", task_name)
    return domain, instance

df = df_csv.copy()

# 1 - split task name into domain and instances
df["benchmark"] = df.reset_index()["task"].map(get_benchmark_labels).values
df["domain"] = df["benchmark"].str.get(0)
df["instance"] = df["benchmark"].str.get(1)
df.drop(columns=["benchmark"], inplace=True)

# 2 - map status from benchexec to integers status
map_status = {
    "true": 1,
    "false": 0,
    "True": 1,
    "False": 0,
    False: 0,
    True: 1,
    "OUT OF MEMORY (false)": -2,
    "TIMEOUT (false)": -1,
    "TIMEOUT (true)": 1,
}
df["status2"] = df["status"].map(map_status)

missing_mapping = df[df["status2"].isnull()].shape[0]
if missing_mapping > 0:
    missing_status = [x for x in df["status"].unique() if x not in map_status.keys()]
    print(f"WARNING: {missing_mapping} status values not mapped:", missing_status)
    print(df[df["status2"].isnull()])

df["status"] = df["status2"]
df.drop(columns=["status2"], inplace=True)

# 3 - define Boolean column solved to flag if solved or not based on status
df.insert(3, "solved", df["status"].apply(lambda x: True if x == 1 else False))


# 4 - extract solver from run name
map_solver = {
    "prp.FOND": "PRP",
    "paladinus.FOND": "PAL",
    "fondsat-glucose.FOND": "FSAT-GL",
    "fondsat-minisat.FOND": "FSAT-MS",
    "cfondasp1-reg.FOND": "ASP1-reg",
    "cfondasp1-fsat.FOND": "ASP1-fsat",
    "cfondasp2-reg.FOND": "ASP2-reg",
    "cfondasp2-fsat.FOND": "ASP2-fsat",
}
df["solver"] = df["run"].map(map_solver)


print("Domains:", df["domain"].unique())
print("Solvers:", df["solver"].unique())

# sanity check status
# df.query("status not in [-1,0,-2,1]")
# df.status = df.status.astype(int) # convert to int
# df.loc[df.status == "OUT OF MEMORY (false)"]
# df.loc[df.status == -1]

# df.dtypes

# note that status should be integer; if float it is bc there must be NaN value!
df

Domains: ['acrobatics' 'beam-walk' 'blocksworld-ipc08' 'blocksworld-new'
 'chain-of-rooms' 'doors' 'earth_observation' 'elevators' 'faults-ipc08'
 'first-responders-ipc08' 'islands' 'miner' 'spiky-tireworld'
 'tireworld-truck' 'tireworld' 'triangle-tireworld' 'zenotravel']
Solvers: ['PAL' 'PRP' 'ASP1-fsat' 'ASP1-reg' 'FSAT-MS' 'FSAT-GL' 'ASP2-fsat'
 'ASP2-reg']


,task,run,status,solved,cputime,walltime,memory_mb,planner_time,policy_size,domain,instance,solver
0,acrobatics_01.yml,paladinus.FOND,1,True,0.416907,0.453278,31.989760,0.036,3,acrobatics,01,PAL
1,acrobatics_02.yml,paladinus.FOND,1,True,0.448627,0.475753,32.026624,0.061,7,acrobatics,02,PAL
2,acrobatics_03.yml,paladinus.FOND,1,True,0.501916,0.543965,32.092160,0.115,15,acrobatics,03,PAL
3,acrobatics_04.yml,paladinus.FOND,1,True,0.681321,0.716574,46.497792,0.261,31,acrobatics,04,PAL
4,acrobatics_05.yml,paladinus.FOND,1,True,1.171665,1.235682,54.849536,0.687,63,acrobatics,05,PAL
...,...,...,...,...,...,...,...,...,...,...,...,...
4715,zenotravel_11.yml,cfondasp2-reg.FOND,-1,False,14400.294209,14404.139162,863.870976,-1.000,-1,zenotravel,11,ASP2-reg
4716,zenotravel_12.yml,cfondasp2-reg.FOND,-1,False,14400.300153,14400.606881,846.143488,-1.000,-1,zenotravel,12,ASP2-reg
4717,zenotravel_13.yml,cfondasp2-reg.FOND,-1,False,14400.297785,14401.525199,762.023936,-1.000,-1,zenotravel,13,ASP2-reg
4718,zenotravel_14.yml,cfondasp2-reg.FOND,-1,False,14400.613112,14402.463129,924.254208,-1.000,-1,zenotravel,14,ASP2-reg


Analyze particular cases:

In [34]:
df.query('solver == "ASP2-reg" and domain == "blocksworld-ipc08"')

,task,run,status,solved,cputime,walltime,memory_mb,planner_time,policy_size,domain,instance,solver
4149,blocksworld-ipc08_01.yml,cfondasp2-reg.FOND,1,True,2.413193,1.804170,64.372736,1.251661,9,blocksworld-ipc08,01,ASP2-reg
4150,blocksworld-ipc08_02.yml,cfondasp2-reg.FOND,1,True,2.121723,1.634624,64.110592,1.108827,9,blocksworld-ipc08,02,ASP2-reg
4151,blocksworld-ipc08_03.yml,cfondasp2-reg.FOND,1,True,4.341235,2.804497,69.296128,2.262615,10,blocksworld-ipc08,03,ASP2-reg
4152,blocksworld-ipc08_04.yml,cfondasp2-reg.FOND,1,True,51.191951,26.987250,100.810752,26.371903,14,blocksworld-ipc08,04,ASP2-reg
4153,blocksworld-ipc08_05.yml,cfondasp2-reg.FOND,1,True,21.589742,11.802611,90.718208,11.231832,13,blocksworld-ipc08,05,ASP2-reg
4154,blocksworld-ipc08_06.yml,cfondasp2-reg.FOND,1,True,7.075828,4.273244,76.206080,3.707095,11,blocksworld-ipc08,06,ASP2-reg
4155,blocksworld-ipc08_07.yml,cfondasp2-reg.FOND,1,True,31.853985,17.023588,89.534464,16.496690,13,blocksworld-ipc08,07,ASP2-reg
4156,blocksworld-ipc08_08.yml,cfondasp2-reg.FOND,1,True,24.285800,13.221974,88.408064,12.669819,13,blocksworld-ipc08,08,ASP2-reg
4157,blocksworld-ipc08_09.yml,cfondasp2-reg.FOND,1,True,2.426874,1.752533,64.458752,1.216284,9,blocksworld-ipc08,09,ASP2-reg
4158,blocksworld-ipc08_10.yml,cfondasp2-reg.FOND,1,True,5.184626,3.399001,75.018240,2.863280,11,blocksworld-ipc08,10,ASP2-reg


Check that solver has been fully extracted (no nulls):

In [35]:
df.query("solver.isnull()")

,task,run,status,solved,cputime,walltime,memory_mb,planner_time,policy_size,domain,instance,solver


Finally, save all results into a complete CSV file. This file can be later used to plot time-coverage graphs as in Nitin's R script.

These tables are not flatten, runs are not across column sets but there is a designated column `solver` that specifies the run of the row.

In [46]:
df.to_csv(os.path.join(CSV_FOLDER, f"{NAME_EXPERIMENT}_benchexec_stats.csv"), index=False)

## 2. Coverage Analysis and Table

We now generate **coverage** tables, as they often appear in papers. Basically, we compute the following per domain and solver-run:

- **Coverage:** % of solved instances solved by the solver-run; and
- **Stat metrics:** mean on time, memory usage, and policy size.

In [36]:
print(df.shape)
df.head()

(4720, 12)


,task,run,status,solved,cputime,walltime,memory_mb,planner_time,policy_size,domain,instance,solver
0,acrobatics_01.yml,paladinus.FOND,1,True,0.416907,0.453278,31.989760,0.036,3,acrobatics,01,PAL
1,acrobatics_02.yml,paladinus.FOND,1,True,0.448627,0.475753,32.026624,0.061,7,acrobatics,02,PAL
2,acrobatics_03.yml,paladinus.FOND,1,True,0.501916,0.543965,32.092160,0.115,15,acrobatics,03,PAL
3,acrobatics_04.yml,paladinus.FOND,1,True,0.681321,0.716574,46.497792,0.261,31,acrobatics,04,PAL
4,acrobatics_05.yml,paladinus.FOND,1,True,1.171665,1.235682,54.849536,0.687,63,acrobatics,05,PAL


Calculate % ratio per set/domain/sub_domain/run-solver.

In [40]:
df_grouped = df.groupby(["domain", "solver"])

#   df_grouped.sum()[["solved"]] = sum all the True instances (sum over bool = number of True)
#   df_grouped.count()[["solved"]] = number of rows in solved column (includes True and False values)
df_coverage = (
    df_grouped.sum(numeric_only=True)[["solved"]] / df_grouped.count()[["solved"]]
)
df_coverage

solved
domain     solver             
acrobatics ASP1-fsat  0.500000
           ASP1-reg   0.500000
           ASP2-fsat  0.500000
           ASP2-reg   0.500000
           FSAT-GL    0.500000
...                        ...
zenotravel ASP2-reg   0.333333
           FSAT-GL    0.333333
           FSAT-MS    0.333333
           PAL        0.333333
           PRP        1.000000

[136 rows x 1 columns]

Calculate mean metric (for CPU time, memory, and policy size) across the solved instances.

In [41]:
columns = ["domain", "solver", "cputime", "memory_mb", "policy_size"]
df_solved = df.query("solved == True")[columns]

df_solved_grouped = df_solved.groupby(["domain", "solver"])
df_metrics = df_solved_grouped.mean()
df_metrics

cputime   memory_mb  policy_size
domain     solver                                         
acrobatics ASP1-fsat   529.188232  166.199296        15.00
           ASP1-reg    796.282176   81.042432        15.00
           ASP2-fsat   427.026727  169.212928        15.25
           ASP2-reg   1595.955871   95.181824        15.25
           FSAT-GL     178.673675   82.584576        15.00
...                           ...         ...          ...
zenotravel ASP2-reg     95.387107  188.794470        12.20
           FSAT-GL     170.958383  466.844058        13.00
           FSAT-MS     406.648567  397.922304        13.00
           PAL          16.789707  368.121446        15.20
           PRP           3.753867   66.439305        54.60

[136 rows x 3 columns]

Put together **Coverage** and **Metrics** tables.

In [43]:
column_names = {
    "solved": "cov",
    "cputime": "time",
    "memory_mb": "mem",
    "policy_size": "size",
}

df_stats = df_coverage.join(df_metrics, how="inner")
df_stats.rename(columns=column_names, inplace=True)

df_stats = df_stats.reset_index()

df_stats

df_stats.query('domain == "blocksworld-ipc08"')

,domain,solver,cov,time,mem,size
16,blocksworld-ipc08,ASP1-fsat,0.333333,18.119117,152.830362,11.200000
17,blocksworld-ipc08,ASP1-reg,0.333333,10.023317,69.580390,11.200000
18,blocksworld-ipc08,ASP2-fsat,0.333333,17.943138,151.499981,11.200000
19,blocksworld-ipc08,ASP2-reg,0.333333,15.248496,78.293402,11.200000
20,blocksworld-ipc08,FSAT-GL,0.400000,343.364766,369.894741,12.166667
21,blocksworld-ipc08,FSAT-MS,0.366667,1031.141888,285.496041,11.636364
22,blocksworld-ipc08,PAL,0.333333,2.256931,88.172954,12.000000
23,blocksworld-ipc08,PRP,1.000000,1.116365,26.998101,24.400000


Finally, pivot the column `solver` into (set of) columns, one per solver.

In [44]:
df_stats_pivot = df_stats.pivot(
    index=["domain"],
    values=["cov", "time", "mem", "size"],
    columns="solver",
)
df_stats_pivot.reset_index(
    inplace=True
)  # unfold multi-index into columns (create integer index)
df_stats_pivot.columns = [
    "_".join(tup).rstrip("_") for tup in df_stats_pivot.columns.values
]

# flat index, but multi-column: 1. coverage / time / policy size and 2. each solver/run
df_stats_pivot = df_stats_pivot.round(2)

df_stats_pivot

,domain,cov_ASP1-fsat,cov_ASP1-reg,cov_ASP2-fsat,cov_ASP2-reg,cov_FSAT-GL,cov_FSAT-MS,cov_PAL,cov_PRP,time_ASP1-fsat,...,mem_PAL,mem_PRP,size_ASP1-fsat,size_ASP1-reg,size_ASP2-fsat,size_ASP2-reg,size_FSAT-GL,size_FSAT-MS,size_PAL,size_PRP
0,acrobatics,0.50,0.50,0.50,0.50,0.50,0.38,1.00,1.00,529.19,...,55.97,12.45,15.00,15.00,15.25,15.25,15.00,9.33,126.50,127.50
1,beam-walk,0.27,0.27,0.27,0.27,0.27,0.27,0.82,1.00,11.53,...,56.60,18.99,18.67,18.67,18.67,18.67,18.67,18.67,453.22,1488.73
2,blocksworld-ipc08,0.33,0.33,0.33,0.33,0.40,0.37,0.33,1.00,18.12,...,88.17,27.00,11.20,11.20,11.20,11.20,12.17,11.64,12.00,24.40
3,blocksworld-new,0.18,0.20,0.18,0.20,0.20,0.20,0.15,1.00,1049.76,...,154.84,264.74,9.14,10.25,9.43,10.50,10.12,10.12,8.00,54.22
4,chain-of-rooms,0.20,0.10,0.20,0.10,0.10,0.10,0.20,1.00,6081.08,...,76.75,11.42,43.00,28.00,43.00,28.00,28.00,28.00,42.00,163.00
5,doors,1.00,1.00,1.00,1.00,1.00,0.87,1.00,0.80,379.66,...,359.96,39.05,19.00,19.00,19.00,19.00,19.00,17.00,17473.73,22.00
6,earth_observation,0.25,0.20,0.25,0.20,0.25,0.15,0.25,1.00,2265.80,...,66.14,10.47,19.20,16.38,19.20,14.88,19.20,11.50,30.60,148.45
7,elevators,0.47,0.47,0.47,0.47,0.53,0.47,0.47,1.00,10.77,...,69.30,10.27,15.86,15.86,15.86,15.86,18.38,15.86,19.71,46.53
8,faults-ipc08,0.71,0.69,0.71,0.67,0.82,0.51,0.85,1.00,779.24,...,564.63,10.44,14.15,13.92,14.15,13.76,13.87,11.25,180.51,15.18
9,first-responders-ipc08,0.45,0.44,0.49,0.50,0.52,0.46,0.30,0.75,986.44,...,510.72,12.33,10.33,9.98,10.92,11.08,11.31,10.24,9.03,18.08


Save coverage table to a CSV file, this can be used in papers.

In [45]:
df_stats_pivot.to_csv(os.path.join(CSV_FOLDER, f"{NAME_EXPERIMENT}_coverage_table.csv"), index=False)